# 사용자 게시글에서 메뉴 확인하기
사용자가 올린 리뷰 게시글에서 명사를 가져와 디저트 사전에 있는 디저트를 사용자가 먹었는지 확인하는 함수생성

In [ ]:
import pandas as pd
import re 
import math
from unicodedata import normalize
import re
import openpyxl
from collections import Counter

In [ ]:
# 한글 자모분리 모듈
!pip install jamo
from jamo import h2j, j2hcj

j2hcj(h2j('ccc'))[0]

'c'

###메뉴사전과 동사사전 가져오기

In [ ]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 아뜰리에 메뉴 사전 dictionary형태로 변환
menu_df = pd.read_csv('메뉴사전.csv', encoding='utf-8')
menu_df = menu_df.fillna("")
menu_df.set_index('Unnamed: 0')

menu_df = menu_df.set_index('Unnamed: 0')
menu_df = menu_df.transpose()

menu_dic = menu_df.to_dict('list')
#''값 제거하기
for i in menu_dic:                              
  menu_dic[i] = list(set(menu_dic[i]))
  if '' in menu_dic[i]:
    menu_dic[i].remove('')

In [ ]:
# 동사사전csv 가져와서 dictionary형태로 구성
verb_df = pd.read_csv('real_동사사전.csv', encoding='utf-8')
verb_df = verb_df.fillna("")
verb_df.set_index('Unnamed: 0')

verb_df = verb_df.set_index('Unnamed: 0')
verb_df = verb_df.transpose()

verb_dic = verb_df.to_dict('list')
#''값 제거하기
for i in verb_dic:                              
  verb_dic[i] = list(set(verb_dic[i]))
  if '' in verb_dic[i]:
    verb_dic[i].remove('')

###사용자 게시글에 디저트가 2개 이상 있는지 확인하는 함수

In [ ]:
def con_menu(post):
  #post내에 있는 nng들을 모두 가지는 nng_list생성
  nng_list=[]
  need_nn = ['NNP','NNG']
  ####nn = ['NNG','NNP','NNB','NP']####
  for word in api.analyze(post):
    for morph in word.morphs:
      if morph.tag in need_nn :
        nng_list.append(morph.lex)
  
  #nng_list내에 디저트 메뉴사전에 있는 단어가 3개 이상이면 nng_count=1, 단어가 하나도 없다면 0으로 break
  count = 0
  nng_count = 0

  ll = len(post)
  if ll > 150 :
    cc = 2
  elif ll<=150 :
    cc = 1

  while True:
    if nng_count >= cc:
      return 1
      break
    elif count >= len(nng_list):
      return 0
      break

    nng_name = nng_list[count]
    nng_first = j2hcj(h2j(nng_name))[0]
    if nng_name in menu_dic[nng_first]:
      nng_count +=1
    count+=1

###사용자 게시글에 먹는것과 관련된 동사가 2개 이상 있는지 확인하는 함수

In [ ]:
def con_verb(post):
  #post내에 있는 동사들을 모두 가지는 verb_list생성
  verb_list=[]
  need_mm = ['VV','VA','VX']
  for word in api.analyze(post):
    for morph in word.morphs:
      if morph.tag in need_mm:
        verb_list.append(morph.lex)
  
  #nng_list내에 디저트 메뉴사전에 있는 단어가 2개 이상이면 nng_count=1, 단어가 하나도 없다면 0으로 break
  count = 0
  verb_count = 0

  ll = len(post)
  if ll > 150 :
    cc = 3
  elif ll<=150 :
    cc = 1
  
  while True:
    if verb_count >= cc :
      return 1
      break
    elif count >= len(verb_list):
      return 0
      break

    verb = verb_list[count]
    verb_first = j2hcj(h2j(verb))[0]
    if verb in verb_dic[verb_first]:
      verb_count +=1
    count+=1

###Hashtag에 있는 메뉴 판별하는 함수

In [ ]:
ll = '개인취향,사가정카페,카페맛집,딸기사랑,휴무스타그램'
l_list = ll.split(",")
l_list

['개인취향', '사가정카페', '카페맛집', '딸기사랑', '휴무스타그램']

In [ ]:
  for word in api.analyze('오늘은 마카롱으로 점심 해결'):
    for morph in word.morphs:
      print(morph.lex, morph.tag)

오늘 NNG
은 JX
마카 NNP
롱 NNG
으로 JKB
점심 NNG
해결 NNG


#최종! con_verb함수와 con_menu를 통해 전체 게시글이 디저트 리뷰인지 구분하는 함수

In [ ]:
def confirm(post):
  result = con_verb(post)*con_menu(post)
  if result == 1:
    return 1
  else:
    return 0

In [ ]:
data = pd.read_csv("row_data_교진.csv",encoding='utf-8').drop("Unnamed: 0",axis=1)
data = data.fillna("NONE")
data.head()

,Nickname,Review,Hashtag
0,chochuchochu,.,"20200613,지아니스나폴리,역삼맛집,역삼나..."
1,a_dcafeartgram,압구정로데오역에 있는 아름다운 스타벅스 리저 매장 :),"스타벅스,스타벅스리저브,압구정로데오카페,카페인테레어,interior,starbuck..."
2,chochuchochu,. . 2020년 2월 1일 토요일 오전 11시30분 충무로 CJ 인재원 그랜드홀,"우리결혼해요,오셔서축하해주세요,결혼,청첩장,wedding"
3,mezas_favorite,. 정말 추천하는 맛집! . . 유튜브 업데이트 됬습니다. 메자스 검색해주세요 .,"다이닝소이,일산맛집,일산카페,정발산카페,정발산맛집,풍산역카페,밤리단길,밤리단길카페,..."
4,ming._.log,""" "" 의 "" "" (￦4,500) . 비엔블랑 가기 전부터 제일 기대했던 갈레트 ...","비엔블랑,갈레트"


dessert Review만을 담을 DataFrame 생성

In [ ]:
desserts = pd.DataFrame(columns=['Nickname','Review','Hashtag'])
desserts.head()

,Nickname,Review,Hashtag


In [ ]:
len(data), len(desserts)

(5636, 9553)

전체 Data 에서 Dessert Review를 걸러내서 Desserts DataFrame에 저장

In [ ]:
for i in range(len(data)) :
  try :
    if len(data.Review[i]) >= 5 :
      if confirm(data.Review[i]) == 1 :
        desserts = desserts.append(data.iloc[[i]],ignore_index=True)
  except :
    continue

In [ ]:
desserts.to_csv('dessert_only.csv',index=False)

from google.colab import files
files.download('dessert_only.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python

from khaiii import KhaiiiApi
api = KhaiiiApi()

In [ ]:
kr='ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅉㅃㅆ'
en='abcdefghijknmlopqrstuvwxyz'
kr_list = [i for i in kr]
en_list = [i for i in en]
all_list = kr_list+en_list

is_kr = re.compile('[^가-힣]')
is_en = re.compile('[-a-zA-Z]')
is_num = re.compile('[0-9]')

In [ ]:
verb_dic = {}
for i in all_list:
  verb_dic[i]=[]

for ii in v_list:
  try:
    first_ii = j2hcj(h2j(ii))[0]
    verb_dic[first_ii].append(ii)
  except:
    continue